In [1]:
import pandas as pd
import ast
from data_process import BERTDataset_For_League
from torch.utils.data import DataLoader
from BERT import BERT,BERTLM, BERTTrainer
from WinnerPredictModel import Winner_Predictor, Winner_Predictor_Trainer
import torch

In [2]:
df = pd.read_csv("../Data/match_data_2.csv")
df["teams"] = df["teams"].apply(lambda x : ast.literal_eval(x))


champ_idx = {}
with open("champ_idx.txt", 'r') as file:
    for line in file:
        pair = ast.literal_eval(line)
        champ_idx.update({pair[0]: pair[1]})



for i in range(len(df["teams"])):
    for j in range(len(df["teams"][i])):
        df["teams"][i][j] = champ_idx[df["teams"][i][j]]


train_datas = [[df["teams"][i][:5], df["teams"][i][5:], df["winner"][i]] for i in range(len(df))]

In [3]:
MAX_LEN = 13
vocab_size = 171

train_data = BERTDataset_For_League(
   train_datas, seq_len=MAX_LEN)

# print(train_data.)

train_loader = DataLoader(
   train_data, batch_size=32, shuffle=True, pin_memory=True)

bert_model = BERT(
  vocab_size=vocab_size,
  d_model= 16,
  n_layers=2,
  heads=8,
  dropout=0.1
)

bert_ = BERTLM(bert_model, vocab_size)
bert_.load_state_dict(torch.load("../bert_model"))




<All keys matched successfully>

In [4]:
wpm = Winner_Predictor(bert_)
wpm_trainer = Winner_Predictor_Trainer(wpm, train_loader, device='cpu')

In [6]:
prev_epochs = 0
epochs = 100
for epoch in range(prev_epochs, epochs):
    wpm_trainer.train(epoch)

EP_train:0:   0%|| 0/938 [00:00<?, ?it/s]


RuntimeError: size mismatch (got input: [2048], target: [32])

In [ ]:
# bert_trainer = BERTTrainer(bert_, train_loader, device='cpu')
